# EDA - Drug Prescription Effectiveness and Ratings
#### Todo:
- explanation of the data and where its from (in the proposal)
- what we're trying to predict
    - i was thinking try to predict a drug's effectiveness / rating based on other reviews
- how were going to do that / what classifiers we use
    - knn
    - decision tree
    - forest (have to use this one)
    - could use others too

#### what gina said we should do - and explain that we need to do this in the EDA too:
- make the class label a high/low rating, instead of 1 to 5
    - or do the PA7 bonus and make the precision and recall scores not binary
- probably take out the usefulCount attribute
- get average scores of each drug and predict on those for each drug
    - then the accuracies will probably be better



you could try to do these rn too but i forget exactly what she said so we can ask her again tomorrow

also the webmd_short.csv is the same thing but with like 1,000 instances instead of 10,000. just bc the other one was taking  
too long to run but we can try to get it to work.

In [38]:
# some useful mysklearn package import statements and reloads
import importlib
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# uncomment once you paste your myclassifiers.py into mysklearn package
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

# get table and header
table, header = myutils.get_tables("input_data/webmd_short.csv")

In [39]:
drug_table = MyPyTable(header, table)
# print(drug_table.column_names)
drug_table.remove_rows_with_missing_values()
drug_table.convert_to_numeric()

# get X and y for stratified k fold cross validation
X = []
y = []
for i in range(len(drug_table.data)):
    # create X, y, remove class label from X
    X.append(drug_table.data[i]) 
    y.append(drug_table.data[i][-1]) # class label
# remove class label from X
for row in X:
    row.pop(-1)


# get folds
X_train_folds, X_test_folds = myevaluation.stratified_kfold_cross_validation(X, y, n_splits=5, random_state=None, shuffle=True)

# create training/testing sets out of the folds
X_train_sets, y_train_sets = myutils.get_sets_from_folds(X, y, X_train_folds)
X_test_sets, y_test_sets = myutils.get_sets_from_folds(X, y, X_test_folds)

labels = [5, 4, 3, 2, 1] # has to be numeric bc the vals in the table are too


# k_neighbors = MyKNeighborsClassifier(n_neighbors=7)
# myutils.fit_predict_classification(X_train_sets, y_train_sets, 
#     X_test_sets, y_test_sets, k_neighbors, "K Neighbors", labels)

dummy = MyDummyClassifier()
myutils.fit_predict_classification(X_train_sets, y_train_sets, 
    X_test_sets, y_test_sets, dummy, "Dummy", labels)

# tree = MyDecisionTreeClassifier()
# myutils.fit_predict_classification(X_train_sets, y_train_sets, 
#     X_test_sets, y_test_sets, tree, "Tree", labels)

Dummy Classification
Accuracy: 0.47 ~ Error Rate: 0.53
Recall Score: 1.0
Precision Score: 0.43
F1 Score: 0.61

Confusion Matrix:
+-----+---+---+---+---+
|  5  | 4 | 3 | 2 | 1 |
+-----+---+---+---+---+
| 286 | 0 | 0 | 0 | 0 |
| 170 | 0 | 0 | 0 | 0 |
| 90  | 0 | 0 | 0 | 0 |
| 48  | 0 | 0 | 0 | 0 |
| 64  | 0 | 0 | 0 | 0 |
+-----+---+---+---+---+
